In [3]:
import os
import concurrent.futures
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv

In [4]:
df = pd.read_csv(r"C:\Users\hp\Desktop\freelance\fb_revit_scraper\fb_bot\fb_group_members.csv")

In [5]:
load_dotenv()
FB_EMAIL = os.getenv('FB_EMAIL')
FB_PASSWORD = os.getenv('FB_PASSWORD')

In [ ]:
df.shape

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from dotenv import load_dotenv

df = pd.read_csv(r"C:\Users\hp\Desktop\freelance\fb_revit_scraper\fb_bot\fb_group_members.csv")
load_dotenv()
FB_EMAIL = os.getenv('FB_EMAIL')
FB_PASSWORD = os.getenv('FB_PASSWORD')

def wait_for_element_presence(driver, xpath, wait_secs=60, by=By.XPATH):
    try:
        WebDriverWait(driver, wait_secs).until(
            EC.presence_of_element_located((by, xpath))
        )
    except Exception as e:
        print(f"wait_for_element_presence Failed {e} XPath :: {xpath}")

def login_to_facebook(driver):
    driver.get('https://www.facebook.com/login')
    wait_after_navigate = '//div[@class="login_form_container"]'
    wait_for_element_presence(driver, wait_after_navigate)
    driver.find_element(By.ID, "email").send_keys(FB_EMAIL)
    driver.find_element(By.ID, "pass").send_keys(FB_PASSWORD)
    driver.find_element(By.NAME, "login").click()
    wait_after_login = '//div[@role="banner"]//div[@aria-label="Facebook" and @role="navigation"]'
    wait_for_element_presence(driver, wait_after_login)
    print("Logged in to Facebook.")

def check_if_message_option_exists(driver, url):
    driver.get(url)
    try:
        message_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//span[text()="Message"]'))
        )
        return True
    except:
        return False

def filter_members_with_message_option(df):
    driver = webdriver.Chrome()
    login_to_facebook(driver)
    
    count = 0
    for index, row in df.iterrows():
        profile_url = row['Member URL']
        if not check_if_message_option_exists(driver, profile_url):
            print(f"Dropped {profile_url}")
            df.drop(index, inplace=True)

        count += 1
        
        # Print progress every 50 profiles
        if count % 50 == 0:
            print(f"Processed {count} profiles...")

        if count % 100 == 0:
            df.to_csv(r"C:\Users\hp\Desktop\freelance\fb_revit_scraper\fb_bot\fb_group_members.csv", index=False)
            print(f"Updated CSV after processing {count} profiles.")

    df.to_csv(r"C:\Users\hp\Desktop\freelance\fb_revit_scraper\fb_bot\fb_group_members.csv", index=False)
    print("Final CSV update completed.")
    
    driver.quit()
    return df

filtered_df = filter_members_with_message_option(df)